In [1]:
%%bash

## Download the training dataset from an INFN archive
wget https://pandora.infn.it/public/269d22/dl/training_set.zip

## Install the unzip utility 
#apt-get -qy install unzip

## Extract the archive
cd $HOME/data/
unzip -qn $HOME/data/training_set.zip

--2024-12-05 15:41:33--  https://pandora.infn.it/public/269d22/dl/training_set.zip
Resolving pandora.infn.it (pandora.infn.it)... 131.154.52.50
Connecting to pandora.infn.it (pandora.infn.it)|131.154.52.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109878633 (105M) [application/force-download]
Saving to: ‘training_set.zip.6’

     0K .......... .......... .......... .......... ..........  0% 60.9M 2s
    50K .......... .......... .......... .......... ..........  0% 93.4M 1s
   100K .......... .......... .......... .......... ..........  0% 67.5M 1s
   150K .......... .......... .......... .......... ..........  0% 85.6M 1s
   200K .......... .......... .......... .......... ..........  0%  107M 1s
   250K .......... .......... .......... .......... ..........  0%  194M 1s
   300K .......... .......... .......... .......... ..........  0%  117M 1s
   350K .......... .......... .......... .......... ..........  0%  102M 1s
   400K .......... .......... ...

In [2]:
import warnings

warnings.filterwarnings('ignore')

from glob import glob
filenames = glob("/home/jovyan/data/data/export/*/*/*/*.png")
print (f"Found {len(filenames)} filenames")

import mlflow
from mlflow.models import infer_signature

Found 612 filenames


In [3]:
learning_rate = 1e-3
batch_size = 10
n_epochs = 50

In [4]:
import mlflow
import os
import tqdm
from tqdm import trange

In [5]:
#username = os.environ['JUPYTERHUB_USER']
#mlflow.set_tracking_uri(f'http://localhost:5000/')
#os.environ["MLFLOW_TRACKING_INSECURE_TLS"] = "true"

# defining a new experiment
#experiment_name = 'MyNewClassifier'
# returns experiment ID
#try:
    # creating a new experiment
#    exp_id = mlflow.create_experiment(name=experiment_name)
#except Exception as e:
#    exp_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

In [6]:
# PREPROCESSING

In [7]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import dask, dask.array

## See Day 2
@dask.delayed 
def load_image(filename: str):
    """Wrapper function loading image as a dask.delayed"""
    return np.asarray(Image.open(filename))

## See Day 2
def load_raw_images(filenames):
    """Load the images from the file paths in `filenames` into a delayed dask-array"""
    return dask.array.stack([
        dask.array.from_delayed(load_image(f), shape=(576, 576), dtype=np.uint8) 
        for f in filenames
    ], axis=0)


## Discussed in Day 1, implemented in Day 2
def windowing(dask_image, x_min, x_max):
    """Maps the pixel values from the interval [x_min, x_max] to [0, 1]"""
    return dask.array.clip((dask_image - x_min)/(x_max - x_min), 0., 1.)

## Discussed in Day 1, implemented in Day 2
def crop_center(dask_image, half_win=64):
    """Crop a numpy-represented image around its center, the resulting image will be a square of side 2*half_win"""
    low, high = 576//2 - half_win, 576//2 + half_win
    return dask_image[:,low:high, low:high]

In [8]:
import re
def energy_keV_from_path(filenames):
    """
    Return a dask array with the energy (in keV) as obtained parsing a sequence of filenames passed 
    as an argument.
    """
    return dask.array.from_array([float(re.findall(r"/([0-9]+)_keV", f)[0]) for f in filenames])

def is_nuclear_from_path(filenames):
    """
    Return an array of boolean, true for nuclear recoil, or false for electron recoils as 
    obtained parsing the list of filenames passed as an argument.
    """
    return dask.array.from_array([float('NR' in re.findall(r"/([NE]R)/", f)) for f in filenames])

In [9]:
shuffled_filenames = np.random.permutation(filenames)

print(filenames[:5])
print(shuffled_filenames[:5])

['/home/jovyan/data/data/export/train/NR/3_keV/cygno-0002.png', '/home/jovyan/data/data/export/train/NR/3_keV/cygno-0001.png', '/home/jovyan/data/data/export/train/NR/20_keV/cygno-0064.png', '/home/jovyan/data/data/export/train/NR/20_keV/cygno-0017.png', '/home/jovyan/data/data/export/train/NR/20_keV/cygno-0063.png']
['/home/jovyan/data/data/export/train/ER/3_keV/cygno-0067.png'
 '/home/jovyan/data/data/export/train/ER/30_keV/cygno-0057.png'
 '/home/jovyan/data/data/export/train/NR/1_keV/cygno-0071.png'
 '/home/jovyan/data/data/export/train/NR/20_keV/cygno-0068.png'
 '/home/jovyan/data/data/export/train/NR/6_keV/cygno-0090.png']


In [10]:
training_files = shuffled_filenames[50:]
validation_files = shuffled_filenames[:50]

In [11]:
training_set = crop_center(windowing(load_raw_images(training_files), 60, 130))
validation_set = crop_center(windowing(load_raw_images(validation_files), 60, 130))

training_label = is_nuclear_from_path(training_files)
validation_label = is_nuclear_from_path(validation_files)

In [12]:
import tensorflow as tf

2024-12-05 15:41:37.635210: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-05 15:41:37.676729: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 15:41:37.676810: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 15:41:37.676835: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 15:41:37.684560: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-05 15:41:37.685742: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [13]:
## The learning rate defines the leap taken at each update of the weights
# learning_rate = 1e-3

## We create a deep sequential model (layers are executed in a sequence, one after the other)
classifier = tf.keras.models.Sequential([
    ## We reshape the images making it explicit that they have 1 single channel
    tf.keras.layers.Reshape((128, 128, 1), name="reshape"),

    ## First Convolutional block (conv + conv + max pooling)
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-2), kernel_initializer='he_normal'),
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-2), kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    
    ## Second Convolutional block (equal to the previous one)
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-2), kernel_initializer='he_normal'),
    tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-2), kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    
    # 3 Conv Layers
    #tf.keras.layers.Conv2D(filters=4, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-2), kernel_initializer='he_normal'),
    #tf.keras.layers.Conv2D(filters=4, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-2), kernel_initializer='he_normal'),
    #tf.keras.layers.MaxPooling2D(pool_size=2),
    
    ## Finally, we flatten the generated image and we use the computed channels as input of a 
    ## last dense layer, activated by a sigmoid.
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='he_normal'),
])

## Before training we may want to specify the input shape of the dataset:
## Batches of any number of elements (None) with each element being a 28x28 matrix
classifier.build(input_shape=(None, 128, 128))

## Then we must assign a loss function and an optimizer to be used 
## for the training.
classifier.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), 
    optimizer=tf.keras.optimizers.Adam(learning_rate)
)

## Optionally, we can print the model obtained, counting the variables and estimating the 
## storage requirements if we plan to export the model.
display(classifier.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 128, 128, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 8)       80        
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 8)       584       
                                                                 
 max_pooling2d (MaxPooling2  (None, 62, 62, 8)         0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 8)         584       
                                                                 
 conv2d_3 (Conv2D)           (None, 58, 58, 8)         584       
                                                        

None

In [14]:
# batch_size = 10

X_train = training_set.rechunk(batch_size)
X_valid = validation_set.rechunk(batch_size)
y_train = training_label.rechunk(batch_size)
y_valid = validation_label.rechunk(batch_size)

In [15]:
#papermill_description=Training
## As above we prepare a list to collect the losses at each step
losses = []
validation_losses = []

# n_epochs = 50
## We count the number of blocks of the X_train 
n_blocks = X_train.numblocks[0]

mlflow.autolog()
## We prepare a progress bar counting the updates
##  (check https://github.com/tqdm/tqdm for docs on progress bars)
    ## We prepare the inner loop, relying on the function zip looping on "pairs" of iterables
    ##   zip([a, b, c], [1, 2, 3]) --> [a, 1], [b, 2], [c, 3] 
with mlflow.start_run(tags={"mlflow.runName": "Train"}) as mlrun:
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", n_epochs)
    mlflow.log_param("lr", learning_rate)
    
    for epoch in range(n_epochs):
        epoch_losses = []
        with trange(n_blocks, unit='updates', desc=f"Epoch {epoch+1}", position=0, leave=False) as progress_bar:
            for image_chunk, label_chunk in zip(X_train.blocks, y_train.blocks):
                progress_bar.update(1)  ## Explicit update of the progress bar
        
                ## We train on a single batch as defined by dask, and store the output in a list
                loss = classifier.train_on_batch(image_chunk, label_chunk)
                epoch_losses.append(loss)
            
            mlflow.log_metric("Train Loss", np.mean(epoch_losses), step=epoch)
            losses.append(np.mean(epoch_losses))
            
            ## We evaluate the trained model on the validation dataset
            validation_loss = classifier.test_on_batch(X_valid, y_valid)
            mlflow.log_metric("Val Loss", validation_loss, step=epoch)
            validation_losses.append(validation_loss)
    mlflow.tensorflow.log_model(classifier, "model")
    classifier.save("classifier.keras")
    mlflow.log_artifact("classifier.keras", artifact_path="model")

2024/12/05 15:41:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2024/12/05 15:41:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


🏃 View run First_Classification_Model at: http://localhost:5000/#/experiments/288674106807194698/runs/c01bd8e245f244e8b2d3a8fe800415d9
🧪 View experiment at: http://localhost:5000/#/experiments/288674106807194698


KeyboardInterrupt: 